In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt') as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        word = word.lower()
        if word in stop_words:
            continue
        word_sorted = ''.join(sorted(word))
        print "%s\t%d\t%s" % (word_sorted, 1, word)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

current_key = None
current_cnt = 0
words_set = set()

for line in sys.stdin:
    try:
        key, cnt, word = unicode(line.strip()).split('\t')
        cnt = int(cnt)
    except ValueError as e:
        continue
    
    if current_key != key:
        if current_key and (len(words_set) > 1):
            print "%d\t%d\t%s" % (current_cnt, len(words_set), ','.join(sorted(words_set)))
        current_key = key
        words_set = set()
        words_set.add(word)
        current_cnt = cnt
    else:
        words_set.add(word)
        current_cnt += cnt
        
print "%d\t%d\t%s" % (current_cnt, len(words_set), ','.join(sorted(words_set)))

Overwriting reducer.py


In [3]:
%%bash

OUT_DIR="word_groups"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming word groups" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
hdfs dfs -cat word_groups/* | grep -P '(,|\t)english($|,)'

7820	5	english,helsing,hesling,shengli,shingle


rm: `word_groups': No such file or directory
19/01/04 21:21:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/04 21:21:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/04 21:21:15 INFO mapred.FileInputFormat: Total input files to process : 1
19/01/04 21:21:15 INFO mapreduce.JobSubmitter: number of splits:2
19/01/04 21:21:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1546618219059_0005
19/01/04 21:21:15 INFO impl.YarnClientImpl: Submitted application application_1546618219059_0005
19/01/04 21:21:15 INFO mapreduce.Job: The url to track the job: http://ed50d717c752:8088/proxy/application_1546618219059_0005/
19/01/04 21:21:15 INFO mapreduce.Job: Running job: job_1546618219059_0005
19/01/04 21:21:22 INFO mapreduce.Job: Job job_1546618219059_0005 running in uber mode : false
19/01/04 21:21:22 INFO mapreduce.Job:  map 0% reduce 0%
19/01/04 21:21:38 INFO mapreduce.Job:  map 13% reduce 0%
19/01/04 21:21:39 INFO mapreduce.Jo